Имеется датасет, содержащий слудующую информацию о пациентах:

`id` - уникальный идентификатор<br>
`age` - возраст<br>
`gender` - пол<br>
`height` - рост<br>
`weight` - вес<br>
`ap_hi` - верхнее давление<br>
`ap_lo` - нижнее давление<br>
`cholesterol` - уровень холестерина<br>
`gluc` - уровень глюкозы<br>
`smoke` - курение<br>
`alco` - алкоголь<br>
`active` - физ. активность<br>
`cardio` - наличие сердечных заболеваний<br>

Необходимо построить модель, которая будет предсказывать наличие сердечных заболеваний. Метрика - ROC AUC.

Импорт необходимых библиотек

In [26]:
from pickle import dump, load

import numpy
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score

from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier

from imblearn.pipeline import Pipeline as imbpipeline

from sklearn.model_selection import train_test_split, cross_val_score

/home/jupyter/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Закрузка датасетов (файлы загружены на vk.com)

In [27]:
data = pd.read_csv('https://vk.com/s/v1/doc/CTTKYV45hY1vHUof4R9w0WZzkjGcV3F_tD_KUb98NXpVn-3wsJo')
test = pd.read_csv('https://vk.com/s/v1/doc/3WmCnNwFN07oEhV4kZ6lUIllsij7UnkUNiFCUv1KAzs57-k5c0s')

Выведем тренировочный датасет.

In [28]:
data.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


Значения в колонке `age` в днях, переведем их в годы.

In [29]:
data['age'] = (data.age / 365)
test['age'] = (test.age / 365)

Описательная статистика.

In [30]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
id,70000.0,49972.419900,28851.302323,0.000000,25006.750000,50001.500000,74889.250000,99999.000000
age,70000.0,53.339358,6.759594,29.583562,48.394521,53.980822,58.430137,64.967123
gender,70000.0,1.349571,0.476838,1.000000,1.000000,1.000000,2.000000,2.000000
height,70000.0,164.359229,8.210126,55.000000,159.000000,165.000000,170.000000,250.000000
weight,70000.0,74.205690,14.395757,10.000000,65.000000,72.000000,82.000000,200.000000
ap_hi,70000.0,128.817286,154.011419,-150.000000,120.000000,120.000000,140.000000,16020.000000
ap_lo,70000.0,96.630414,188.472530,-70.000000,80.000000,80.000000,90.000000,11000.000000
cholesterol,70000.0,1.366871,0.680250,1.000000,1.000000,1.000000,2.000000,3.000000
gluc,70000.0,1.226457,0.572270,1.000000,1.000000,1.000000,1.000000,3.000000
smoke,70000.0,0.088129,0.283484,0.000000,0.000000,0.000000,0.000000,1.000000


Удалим выбросы в колонках `height` и `weight`, удалив по 1% значений.

In [31]:
print(data.height.quantile(0.01))
print(data.height.quantile(0.99))

147.0
184.0


In [32]:
data = data.loc[(data.height > 147) & (data.height < 184)]

In [33]:
print(data.weight.quantile(0.01))
print(data.weight.quantile(0.99))

48.0
117.0


In [34]:
data = data.loc[(data.weight > 45) & (data.weight < 120)]

Код ниже очищает колонки, содержащие данные о давлении от отрицательных и аномальных значений. Часть значений были введены с ошибками - их необходимо умнижить или поделить на 10 (некоторые по 2 раза).

In [35]:
data.loc[data.ap_hi < 0, 'ap_hi'] = data.loc[data.ap_hi < 0, 'ap_hi'] * -1
data.loc[(data.ap_hi >= 10) & (data.ap_hi <= 20), 'ap_hi'] = data.loc[(data.ap_hi >= 10) & (data.ap_hi <= 20), 'ap_hi'] * 10
data = data.loc[~(data.ap_hi < 25)]
data.loc[data.ap_hi >= 300, 'ap_hi'] = (data.loc[data.ap_hi >= 300, 'ap_hi'] / 10).round(0).astype('int')
data.loc[data.ap_hi >= 300, 'ap_hi'] = (data.loc[data.ap_hi >= 300, 'ap_hi'] / 10).round(0).astype('int')
data = data.loc[~(data.ap_lo < 50)]
data.loc[data.ap_lo > 180, 'ap_lo'] = (data.loc[data.ap_lo > 180, 'ap_lo'] / 10).round(0).astype('int')
data.loc[data.ap_lo > 180, 'ap_lo'] = (data.loc[data.ap_lo > 180, 'ap_lo'] / 10).round(0).astype('int')
data = data.loc[~(data.ap_lo < 45)]

Многие значения в колонках верхнее давление - вес, нижнее давление - рост совпадают, заменим их на медианные.

In [36]:
data.loc[(data.weight == data.ap_lo), ['weight', 'ap_lo']] = numpy.nan

In [37]:
data['weight'] = data['weight'].fillna(data.groupby(['gender', pd.qcut(data.height, q=10, precision=0)])['weight'].transform(lambda x : x.median()))
data['ap_lo'] = data['ap_lo'].fillna(data.groupby(['gender', pd.qcut(data.ap_hi, q=10, precision=0, duplicates='drop')])['ap_lo'].transform(lambda x : x.median()))

In [38]:
data.loc[(data.height == data.ap_hi), ['height', 'ap_hi']] = numpy.nan

In [39]:
data['height'] = data['height'].fillna(data.groupby(['gender', pd.qcut(data.weight, q=10, precision=0)])['height'].transform(lambda x : x.median()))
data['ap_hi'] = data['ap_hi'].fillna(data.groupby(['gender', pd.qcut(data.ap_lo, q=10, precision=0, duplicates='drop')])['ap_hi'].transform(lambda x : x.median()))

Нижнее давление не должно быть выше верхнего - поменяем значения местами.

In [40]:
reversed = data.loc[data.ap_lo > data.ap_hi, ['ap_hi', 'ap_lo']].copy()
reversed

,ap_hi,ap_lo
474,120.0,150.0
636,70.0,110.0
2384,90.0,150.0
2990,80.0,140.0
3447,80.0,125.0
...,...,...
66315,100.0,160.0
66657,80.0,120.0
67421,80.0,130.0
67470,80.0,120.0


In [41]:
data.loc[reversed.index, 'ap_lo'] = reversed['ap_hi']
data.loc[reversed.index, 'ap_hi'] = reversed['ap_lo']

Создаим колонку `ap_rate` для расчета отношения верхнего давления к нижнему и исключим наиболее подозрительные значения по этому показателю.

In [42]:
data['ap_rate'] = data['ap_lo'] / data['ap_hi']

In [43]:
data = data.loc[~(data.ap_rate >= 0.9)]

Удалим неиформативные столбцы.

In [44]:
data = data.drop(['id','ap_rate'], axis=1)

Разделим датасет.

In [45]:
X_train = data.drop('cardio', axis=1)
y_train = data['cardio']

In [46]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67303 entries, 0 to 69999
Data columns (total 11 columns):
age            67303 non-null float64
gender         67303 non-null int64
height         67303 non-null float64
weight         67303 non-null float64
ap_hi          67303 non-null float64
ap_lo          67303 non-null float64
cholesterol    67303 non-null int64
gluc           67303 non-null int64
smoke          67303 non-null int64
alco           67303 non-null int64
active         67303 non-null int64
dtypes: float64(5), int64(6)
memory usage: 6.2 MB


Напишем функцию для подбора гиперпараметров.

In [47]:
def GBC(n_of_iterations):
    preprocessor = ColumnTransformer(
        transformers=[('scaler', StandardScaler(), ['height',                        #количественные признаки
                                                    'weight',
                                                    'age',
                                                    'ap_hi',
                                                    'ap_lo']),
                     ('ordinal_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1),
                                                           ['gender',                #категориальные признаки
                                                            'cholesterol',
                                                            'gluc'])],
        remainder='passthrough'
    )
    
    pipeline_gbc = imbpipeline([
        ('preprocessor', preprocessor),
        ('gbc', GradientBoostingClassifier(random_state=100))
    ])

    params_gbc = {
        'gbc__n_estimators':[100, 125],
        'gbc__subsample':[0.8, 0.9],
        'gbc__max_depth':range(3, 5),
        'gbc__max_features':range(5, 10),
        'gbc__min_samples_leaf': range(1, 21, 3)
    }
    
    search = RandomizedSearchCV(pipeline_gbc, params_gbc, n_iter=n_of_iterations, cv=4, scoring='roc_auc')
    search.fit(X_train, y_train)
    results = search.cv_results_
    return search.best_score_, search.best_params_

In [48]:
#!c1.32
%%time
roc_auc, best_params = GBC(1)

print('ROC_AUC on VALID:', '%.4f' %roc_auc)
print(pd.Series(best_params))

/home/jupyter/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ROC_AUC on VALID: 0.8029
gbc__subsample             0.8
gbc__n_estimators        125.0
gbc__min_samples_leaf      7.0
gbc__max_features          9.0
gbc__max_depth             4.0
dtype: float64
CPU times: user 27.7 s, sys: 2.59 s, total: 30.3 s
Wall time: 30.2 s


Лучшие результаты (100 итераций):

ROC_AUC on VALID: 0.8030<br>
gbc__subsample              0.9<br>
gbc__n_estimators         100.0<br>
gbc__min_samples_split     15.0<br>
gbc__min_samples_leaf      13.0<br>
gbc__max_features           6.0<br>
gbc__max_depth              4.0<br>

In [49]:
preproc = ColumnTransformer(
    transformers=[('scaler', StandardScaler(), ['age',
                                                'height',
                                                'weight',
                                                'ap_hi',
                                                'ap_lo']),
                  ('ordinal_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1),
                                               ['gender',
                                                'cholesterol',
                                                'gluc'])],
    remainder='passthrough'
)
    
pipe = Pipeline([
    ('preprocessor', preproc),
    ('gbc', GradientBoostingClassifier(random_state=100,
                                       n_estimators=100,
                                       min_samples_leaf=13,
                                       max_depth=4,
                                       subsample=0.9,
                                       min_samples_split=15,
                                       max_features=6))
])

pipe.fit(X_train, y_train)

/home/jupyter/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', StandardScaler(),
                                                  ['age', 'height', 'weight',
                                                   'ap_hi', 'ap_lo']),
                                                 ('ordinal_encoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['gender', 'cholesterol',
                                                   'gluc'])])),
                ('gbc',
                 GradientBoostingClassifier(max_depth=4, max_features=6,
                                            min_samples_leaf=13,
                                            min_samples_split=15,
                                            random_state=100, subsample=0.9))])

Подготовим тестовый датасет.

In [50]:
X_test = test.drop(['id'], axis=1)

In [51]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 11 columns):
age            30000 non-null float64
gender         30000 non-null int64
height         30000 non-null int64
weight         30000 non-null float64
ap_hi          30000 non-null int64
ap_lo          30000 non-null int64
cholesterol    30000 non-null int64
gluc           30000 non-null int64
smoke          30000 non-null int64
alco           30000 non-null int64
active         30000 non-null int64
dtypes: float64(2), int64(9)
memory usage: 2.5 MB


In [53]:
predictions = pipe.predict_proba(X_test)

In [54]:
predictions[:,1]

array([0.49429452, 0.56041732, 0.40137258, ..., 0.45561181, 0.27814941,
       0.70842908])

Подготовим датафрейм для закрузки на Kaggle.

In [55]:
results = pd.concat([test['id'], pd.Series(predictions[:,1])], axis=1)

In [56]:
results = results.rename(columns={0: 'cardio'})

In [57]:
results

,id,cardio
0,5,0.494295
1,6,0.560417
2,7,0.401373
3,10,0.562267
4,11,0.229546
...,...,...
29995,99984,0.872072
29996,99987,0.195398
29997,99989,0.455612
29998,99994,0.278149


In [58]:
results.to_csv('results.csv', encoding='utf-8', index=False)

Сохранение модели

In [ ]:
#with open('model2.pcl', 'wb') as fid:
#    dump(pipe, fid)